# Finite Difference Method for the Wave Equation on the Unit Interval

Here we solve the wave equation $\frac{\partial^2 \, u}{\partial t^2} = c^2 \frac{\partial^2 \, u}{\partial x^2}$ on the spatial domain $x \in \Omega=(0,1)$ and time domain $t \in [0,T]$ supplied with homogeneous Dirichlet boundary conditions and initial conditions for displacement and velocity. For the spatial discretization, the finite difference method on an uniform mesh is employed. For the time discretization, a fixed time step is employed.  

## Import Packages

In [ ]:
using LinearAlgebra
using DifferentialEquations
using SparseArrays
using Plots
using BenchmarkTools 

Documentation of DifferentialEquations.jl  
<ul>
<li> <a href="https://diffeq.sciml.ai/dev/tutorials/ode_example/"> Examples </a>   
<li> <a href="https://diffeq.sciml.ai/stable/"> Documentation of DifferentialEquations.jl on Github </a>  
<li> <a href="https://diffeq.sciml.ai/dev/features/performance_overloads/#Function-Type-Definitions"> Specifying Jacobian types using DifferentialEquations.jl </a>   
</ul>   

## Section 1: Construct Traveling Gauss-Pulse as a Reference Analytical Solution + plot

In this part, we construct a d'Alembert reference solution using method of characteristics. This method is explained and illustrated e.g. in the slides  [http://www-thphys.physics.ox.ac.uk/people/FrancescoHautmann/Cp4p/sl_wv_3pp_12.pdf]. This solution assumes the domain in space to be unbounded. The reflection from the boundaries are *not* taken into account. An analytical reference solution is valuable to have a reference for the subsequent numerical solution.    

In [ ]:
#..Define analytical reference solution
#..We choose as reference solution a Gauss pulse travelling to the right
#..The part of the d'Alembert solution traveling to the right is not taken into account  
#..c:  velocity of propagation 
#..x0: center of the Gauss pulse 
#..sigma: width of the Gauss pulse 
function gauss_pulse(x,t,c,x0,sigma)
    xi = x - c*t; 
    return exp(-(xi-x0)^2/sigma^2)
end

function ddt_gauss_pulse(x,t,c,x0,sigma)
    # d/dt [f(xi)] = d/dt[exp(-(xi-x0)^2)/sigma^2] = [exp(-(xi-x0)^2)/sigma^2]*[-2*(xi-x0)/sigma^2]*[d xi / dt]
    # = [exp(-(xi-x0)^2)/sigma^2]*[-2*(xi-x0)/sigma^2]*[-c]
    xi = x - c*t;
    return exp(-(xi-x0)^2/sigma^2)*(-2*(xi-x0)/sigma^2)*(-c)
end

#..construct the 1D mesh
nelements = 100; nnodes = nelements+1; h = 1/nelements; h2=h*h;
N = 100; Np1 = N+1; h = 1/N; h2=h*h; 
x = Vector(0:h:1); 

#..temporal mesh and set the velocity  
T = 10; Nt = 10000; dt = T/Nt; 
t = Vector(0:dt:T); 
c = .2; #..wave velocity 

#..Evaluate the initial condition  
#..hurray for broadcast in Julia 
u0 = gauss_pulse.(x,0,c,0.3,0.1)
u1 = gauss_pulse.(x,1,c,0.3,0.1)
u2 = gauss_pulse.(x,2,c,0.3,0.1)
u3 = gauss_pulse.(x,3,c,0.3,0.1)

#..Evaluate initial velocity 
v0 = ddt_gauss_pulse.(x,0,c,0.3,0.1)

#..Evaluate the analytical solution in space-time domain
#..hurray for comprehension 
Uex = [gauss_pulse(xi,tk,.01,0.3,0.1) for  xi in x, tk in t]

#..plot the initial solution  
p1=plot(x,u0,lw=2,legend=false)
xlabel!("x") 
ylabel!("u_0(x)")
p2=plot(x,v0,lw=2,legend=false)
xlabel!("x") 
ylabel!("v_0(x)")
p3=plot(x,u2,lw=2,legend=false)
xlabel!("x") 
ylabel!("u_2(x)")
p4=plot(x,u3,lw=2,legend=false)
xlabel!("x") 
ylabel!("u_3(x)")

#..plot the initial solution  
p5=contour(t,x,Uex)
xlabel!("t") 
ylabel!("x")
#..plot the initial solution  
p6=surface(t,x,Uex)
xlabel!("t") 
ylabel!("x")

p7=plot(p1, p2, p3, p4, p5, p6, layout = (3, 2))

## Section 2: Spatial Discretization: Construct the One-Dimensional Matrix

In [ ]:
# struct to hold entire mesh
struct Mesh
  #..number of nodes of the mesh..   
  nnodes::Int64
  #..number of elements of the mesh..
  nelements::Int64 
  #..vector of x-coordinates of nodes of the mesh..
  Nodes::Vector{Float64}
end 

In [ ]:
# function to generate a mesh on the interval 0 <= x <= 1.   
# we limit the type of input to be Int64 
function genMesh(nelements::Int64)::Mesh
    h = 1/nelements 
    nnodes = nelements+1
    Nodes = Vector{Float64}(0:h:1)    
    mesh = Mesh(nnodes,nelements,Nodes)     
    return mesh;
end 

# generate global stiffness matrix 
function genStiffMat(mesh::Mesh)

    #..recover number of elements and nodes  
    nelements = mesh.nelements
    nnodes    = nelements+1
    
    #..set mesh width and square of mesh width 
    h = 1/nelements; h2=h*h; 

    #..construct the coefficient matrix with our the boundary conditions 
    e = ones(nnodes); #..note that ones(N+1,1) does *not* work here 
    A = Tridiagonal(-e[2:end], 2*e, -e[2:end]); 
    A = (1/h2)*A;     
   
    return A; 
    
end

function genVector(mesh, sourceFct::F) where F 

    #..recover mesh nodes  
    nodes = mesh.Nodes
    
    #..set vector values using broadcasting (dot syntax)  
    f = sourceFct.(nodes)
   
    return f; 
end

function genSolution(A,f)
    #..handle essential boundary conditions 
    A[1,1] = 1.; A[1,2] = 0.; f[1] = 0.;
    A[end,end] = 1.; A[end,end-1] = 0.; f[end] = 0.;
    u = A\f 
    return u 
end

## Section 3: Implicit Integration Using Home-Brewed Code and Fixed Time Step 

Here we perform implicit time-integration using a home-brewed code and a fixed time step. Here we cheat and use the d'Alembert solution for the first two time steps. We do so an an intermediate step. 

Implicit time integration 

Na discretisatie in de ruimte: \vec{u}_tt = (c^2) A \vec{u} with A having *negative* diagonal elements 

Na discretisation in cde tijd: \vec{u}^{k-1} - 2\vec{u}^k + \vec{u}^{k+1} =  (dt*c)^2 (A) \vec{u}^{k+1}

of [ I - (dt*c)^2 A] \vec{u}^{k+1} = 2\vec{u}^k  - \vec{u}^{k-1} 

Time advancement: B \vec{u}^{k+1} = 2 \vec{u}^k  - vec{u}^{k-1} where B = I - (dt*c)^2 A 

In [ ]:
nelements = 100; nnodes = nelements+1;
mesh = genMesh(nelements); A = -genStiffMat(mesh);

In [ ]:
#..perform implicit time integration 
U = zeros(Np1,Nt+1)
U[:,1] = Uex[:,1];
U[:,2] = Uex[:,2];
B = Diagonal(ones(nnodes)) - (dt*c)^2*A
B[1,1] = 1; B[1,2] = 0; B[end,end]=1; B[end,end-1]=0; 
for k in 3:Nt
  rhs = 2*U[:,k-1] - U[:,k-2]
  U[:,k] = B \ rhs; 
end 
p1 = contour(t,x,U)
p2 = plot(x,U[:,1])
p3 = plot(x,U[:,1000])
p4 = plot(x,U[:,2000])
p5 = plot(x,U[:,5000])
p6 = plot(x,U[:,9000])
plot(p1,p2, p3,p4,p5,p6,layout=(3,2))

## Section 4: Explicit Time Integration using Home-Brewed Code and Fixed Time Step  

Here we perform explicit time-integration using a home-brewed code and a fixed time step. Here we cheat and use the d'Alembert solution for the first two time steps. We do so an an intermediate step. 

Explicit time integration - how to handle the boundary conditions?  

Na discretisatie in de ruimte: \vec{u}_tt = (c^2) A \vec{u} with A having *negative* diagonal elements

Na discretisation in cde tijd: \vec{u}^{k-1} - 2\vec{u}^k + \vec{u}^{k+1} =  (dt*c)^2 (A) \vec{u}^{k}

A met negatieve elementen op de diagonaal 

of \vec{u}^{k+1} = 2\vec{u}^k  - \vec{u}^{k-1} + (dt*c)^2 (A) \vec{u}^{k}

In [ ]:
#..perform explicit time integration 
U = zeros(Np1,Nt+1)
U[:,1] = Uex[:,1];
U[:,2] = Uex[:,2];
B[1,1] = 1; B[1,2] = 0; B[end,end]=1; B[end,end-1]=0; 
for k in 3:Nt
  rhs = 2*U[:,k-1] - U[:,k-2] + (dt*c)^2*A*U[:,k-1]
  U[:,k] = B \ rhs; 
end 
p1 = contour(t,x,U)
p2 = plot(x,U[:,1])
p3 = plot(x,U[:,1000])
p4 = plot(x,U[:,2000])
p5 = plot(x,U[:,5000])
p6 = plot(x,U[:,9000])
plot(p1,p2, p3,p4,p5,p6,layout=(3,2))

## Section 5: Plug and Play using DifferentialEquations.jl   

In [ ]:
function wave_system!(du,u,p,t)
    A*u
end

u0 = gauss_pulse.(x,0,c,0.3,0.1)
v0 = ddt_gauss_pulse.(x,0,c,0.3,0.1)

tspan = (0.0,5.0)               

prob = SecondOrderODEProblem(wave_system!,v0,u0,tspan)
sol = solve(prob)

p1=plot(sol)

In [ ]:
plot(sol.u[100,:])

## References